# Apparels Image Classification using TensorFlow 2.x

[source](https://www.analyticsvidhya.com/blog/2020/03/tensorflow-2-tutorial-deep-learning/)

We will train a neural network model to classify images of clothing, like sneakers and shirts.

We are using tf.keras, the high-level API to build and train models in TensorFlow. 

## Upgrade pip

In [23]:
! pip install --upgrade pip

     |████████████████████████████████| 1.5MB 18.8MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3


## Install Necessary Libraries including TensorFlow and Keras

### Troubleshoot "Could not import PIL.Image" Issue

Install pillow

In [1]:
# ! pip uninstall  -y pillow

In [2]:
! pip install pillow

Check if both of below imports pointing to the same python3 directory

In [3]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [4]:
from PIL import Image
print(Image.__file__)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py


In [5]:
import Image
print(Image.__file__)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py


If both import prints pointing to the same python3 directory, install tensorflow

In [6]:
! pip install numpy pandas sklearn matplotlib tqdm tensorflow==2.0.0

## Import Necessary Libraries including TensorFlow and Keras

In [7]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [8]:
print(tf.__version__)

2.0.0


In [9]:
print(keras.__version__)

2.2.4-tf


In [10]:
image=keras.preprocessing.image

In [11]:
to_categorical=keras.utils.to_categorical

In [12]:
# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

## Import the Apparels Data

Download the train and test data from [the practice problem page](https://datahack.analyticsvidhya.com/contest/practice-problem-identify-the-apparels/#ProblemStatement) and then upload to storage/ directory.

```
! apt-get update
! apt-get upgrade
! apt-get install -y wget
! apt-get install -y zip unzip
```

In [13]:
# ! unzip storage/train.zip
# ! unzip storage/test.zip

We have total 70,000 images (28 x 28), out of which 60,000 are part of train images with the label of the type of apparel (total classes: 10) and rest 10,000 images are unlabelled (known as test images).The task is to identify the type of apparel for all test images. Given below is the code description for each of the apparel class/label.

```
Label	Description
0	T-shirt/top
1	Trouser
2	Pullover
3	Dress
4	Coat
5	Sandal
6	Shirt
7	Sneaker
8	Bag
9	Ankle boot
```

In [14]:
train = pd.read_csv('train.csv')

In [15]:
train

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0
...,...,...
59995,59996,5
59996,59997,1
59997,59998,3
59998,59999,0


In [16]:
train.shape

(60000, 2)

In [41]:
type(train)

pandas.core.frame.DataFrame

## Preprocess The Data

We will import the data and preprocess it. This includes reading all the images from the train folder one by one and then doing some necessary preprocessing steps such as dividing by 255 to bring all values between 0 and 1. We would also need to convert the target to categorical as right now they are numerical in form and the model needs to understand these as categories.

In [42]:
# We have grayscale images, so while loading the images we will set color_mode to grayscale, if you have RGB images, you should set color_mode to rgb
# see https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/load_img
def load_grayscale_images(df, image_path):
    images = []
    for i in tqdm(range(df.shape[0])):
        img = image.load_img(image_path+df['id'][i].astype('str')+'.png', target_size=(28,28,1), color_mode="grayscale")
        img = image.img_to_array(img)
        img = img/255
        images.append(img)
    return np.array(images)    

In [43]:
X = load_grayscale_images(train, 'train/')

100%|██████████| 60000/60000 [00:18<00:00, 3226.56it/s]


In [44]:
X.shape

(60000, 28, 28, 1)

In [45]:
# Preprocessing the Target
y=train['label'].values
y = to_categorical(y)

In [46]:
y.shape

(60000, 10)

In [47]:
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Build the Deep Learning Model

Now that we have preprocessed the images and labels, it is time to define the model. Here, we are using a convolutional neural network (CNN) model. For those of you who are new to CNNs, I encourage you to go through this [excellent tutorial](https://www.analyticsvidhya.com/blog/2017/06/architecture-of-convolutional-neural-networks-simplified-demystified/?utm_source=blog&utm_source=tensorflow-2-tutorial-deep-learning). Before we declare the model, we will split the train data into new train and validation sets in order to check performance at each epoch:

In [48]:
# Create Train and validation data to check the performance at each epoch
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [49]:
X_train.shape

(48000, 28, 28, 1)

In [50]:
X_test.shape

(12000, 28, 28, 1)

In [51]:
y_train.shape

(48000, 10)

In [52]:
y_test.shape

(12000, 10)

In [66]:
# Using Keras Sequential API to add neural network layers
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

We have chosen the above architecture iteratively after trying various hyperparameters to get better accuracy. You can go through some tips and tricks to improve your model performance at this [link](https://www.analyticsvidhya.com/blog/2019/11/4-tricks-improve-deep-learning-model-performance/?utm_source=blog&utm_source=tensorflow-2-tutorial-deep-learning).

## Compile & Train the Deep Learning Model

Once we have defined the neural network architecture we will now compile it and train the model to check its performance on the validation set:

In [67]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 43s 899us/sample - loss: 0.5054 - accuracy: 0.8208 - val_loss: 0.3159 - val_accuracy: 0.8838
Epoch 2/20
48000/48000 [==============================] - 43s 887us/sample - loss: 0.3369 - accuracy: 0.8784 - val_loss: 0.2625 - val_accuracy: 0.9018
Epoch 3/20
48000/48000 [==============================] - 44s 911us/sample - loss: 0.2868 - accuracy: 0.8949 - val_loss: 0.2463 - val_accuracy: 0.9087
Epoch 4/20
48000/48000 [==============================] - 44s 912us/sample - loss: 0.2525 - accuracy: 0.9071 - val_loss: 0.2267 - val_accuracy: 0.9173
Epoch 5/20
48000/48000 [==============================] - 41s 859us/sample - loss: 0.2287 - accuracy: 0.9161 - val_loss: 0.2177 - val_accuracy: 0.9199
Epoch 6/20
48000/48000 [==============================] - 41s 855us/sample - loss: 0.2065 - accuracy: 0.9230 - val_loss: 0.2244 - val_accuracy: 0.9182
Epoch 7/20
48000/48000 [====================

Wow, this model is already giving great accuracy (0.92) at the validation set.

We can do more experiments such as:
- Increasing epochs
- Use more layers

This will help you to get an even better score on the validation set.

## Make Predictions & Evaluate Accuracy

In [57]:
# Read test file names
test = pd.read_csv('test.csv')

In [58]:
type(test)

pandas.core.frame.DataFrame

In [62]:
test

,id
0,60001
1,60002
2,60003
3,60004
4,60005
...,...
9995,69996
9996,69997
9997,69998
9998,69999


In [64]:
len(test['id'])

10000

In [59]:
# Read test images and preprocess
test_X = load_grayscale_images(test, 'test/')

100%|██████████| 10000/10000 [00:03<00:00, 3264.70it/s]


In [68]:
# Make Predictions using the trained model
test_prediction = model.predict_classes(test_X)

In [69]:
len(test_prediction)

10000

In [72]:
submission = pd.DataFrame({})

In [74]:
submission.empty

True

In [75]:
# Write submission file to score
submission['id'] = test['id']
submission['label'] = test_prediction
submission.to_csv('submission_cnn.csv', header=True, index=False)

Once this submission is created, you can download it from the left-hand side pane in the Notebook and upload it at the solution checker to check the accuracy score for the test set. Right now with the above code, you will get a score of 0.922 on the public leaderboard.